In [ ]:
%matplotlib inline


Задача переноса стиля на изображении
=============================

За основу данного семинара взят ноутбук https://pytorch.org/tutorials/advanced/neural_style_tutorial.html 

Введение
------------

В этом ноутбуке мы реализуем алгоритм переноса стиля из статьи https://arxiv.org/abs/1508.06576 Leon A. Gatys, Alexander S. Ecker, Matthias Bethge.

Перенос стиля (Neural-Style, Neural-Transfer) --- это процесс преобразования стиля исходного изображения к стилю выбранного изображения.

***Основной идеей алгоритма, описанного в ноутбуке, является то, что
признаки, полученные с помощью сверточной нейронной сети, можно использовать
для выделения содержательной и стилевой составляющих изображения.***

На вход алгоритму требуется три изображения:
* input-image - изменяемое входное изображение (обычно берется исходное изображение или белый шум)
* content-image - исходное изображение

* style-image - изображение, стиль которого будет использован


Алгоритм изменяет input-image таким образом, чтобы содержание было похоже на content-image, а стиль - на style-image.


Формализация идеи
--------------------

Мы определяем два расстояния (метрики):
* $D_C$ - расстояние между содержанием на изображениях (content distance)
* $D_S$ - расстояние между стилями на изображениях (style distance)

Затем берём изменяемое изображение (input-image) и трансформируем его так, чтобы на результирующем изображении минимизровались оба расстояния.


Импортируем необходимые библиотеки
-----------------------------------------

-  ``torch``, ``torch.nn``, ``numpy``
-  ``torch.optim``
-  ``PIL``, ``PIL.Image``, ``matplotlib.pyplot`` (загрузка и визуализация изображений)
-  ``torchvision.transforms`` (трансформирует PIL-изображения в тензоры)
-  ``torchvision.models`` 
-  ``copy`` 



In [ ]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

import copy

Если возможно - запускаем алгоритм на GPU для ускорения вычислений:
``torch.cuda.is_available()``



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Загружаем изображения

------------------

Загружаем стилевое изображение и изменяемое (content) изображение.

Исходные PIL-картинки содержат значения от 0 до 255, однако при их преобразовании в тензоры значения переходят на отрезок от 0 до 1.

Также изображения должны быть приведены к одинаковым размерам.

***Важно!*** В pytorch значения тензоров должны лежать на отрезке от 0 до 1. Если попробовать применить модель к тензору со значениями от 0 до 255, то в результате применения свёрток (feature maps) не получится корректно выделить содержание и стиль на изображении.

In [ ]:
# desired size of the output image
imsize = 512 if torch.cuda.is_available() else 128  # use small size if no gpu

loader = transforms.Compose([
    transforms.Resize(imsize),  # scale imported image
    transforms.ToTensor()])  # transform it into a torch tensor


def image_loader(image_name):
    image = Image.open(image_name)
    # fake batch dimension required to fit network's input dimensions
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)

!wget https://pytorch.org/tutorials/_static/img/neural-style/picasso.jpg
!wget https://pytorch.org/tutorials/_static/img/neural-style/dancing.jpg

style_img = image_loader("picasso.jpg")
content_img = image_loader("dancing.jpg")

assert style_img.size() == content_img.size(), \
    "we need to import style and content images of the same size"

Создадим функцию, которая визуализирует изображение, копируя его и переводя обратно в PIL формат.


In [ ]:
unloader = transforms.ToPILImage()  # reconvert into PIL image

plt.ion()

def imshow(tensor, title=None):
    image = tensor.cpu().clone()  # we clone the tensor to not do changes on it
    image = image.squeeze(0)      # remove the fake batch dimension
    image = unloader(image)
    plt.imshow(image)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) # pause a bit so that plots are updated


plt.figure()
imshow(style_img, title='Style Image')

plt.figure()
imshow(content_img, title='Content Image')

Функции потерь (Loss)
--------------

***Content Loss***

Content loss - взвешенный вариант content distance ($D_C$) для каждого свёрточного слоя сети.
* принимает на вход feature maps $F_{XL}$ для изображения $X$ и слоя $L$ сети
* возвращает взвешенное рассстояние $w_{CL}.D_C^L(X,C)$ между изображением $X$ и контентным изображением $C$.

Feature maps контентного изображения ($F_{CL}$) должны быть известны заранее для вычисления контентного расстояния:
$$D_C^L(X,C)=\|F_{XL} - F_{CL}\|^2$$ (для вычисления используем ``nn.MSELoss``).


In [ ]:
class ContentLoss(nn.Module):

    def __init__(self, target,):
        super(ContentLoss, self).__init__()
        # we 'detach' the target content from the tree used
        # to dynamically compute the gradient: this is a stated value,
        # not a variable. Otherwise the forward method of the criterion
        # will throw an error.
        self.target = target.detach()

    def forward(self, input):
        self.loss = F.mse_loss(input, self.target)
        return input

**Style Loss**

Модуль Style loss выглядит похожим образом.

Чтобы вычислить style loss, необходимо вычислить матрицу Грама (матрицу скалярных произведений) $G_{XL}$. Матрица Грама - это результат умножения матрицы на её транспонированный вариант. 

В данном случае нам нужна матрица feature maps $F_{XL}$ для слоя $L$. Мы трансформируем матрицу $F_{XL}$, чтобы она имела размерность $K \times N$, где 
* $K$ - число feature maps на слое $L$
* $N$ - длина каждой вытянутой в вектор feature map $F_{XL}^k$.
Обозначим матрицу требуемой размерности $\hat{F}_{XL}$.

**Пояснение:**
например, первая строка матрицы $\hat{F}_{XL}$ соответствует первой feature map $F_{XL}^1$, вытянутой в вектор.

Наконец, мы нормализуем матрицу Грама путем деления каждого элемента на число элементов матрицы. 

**Зачем это нужно?**
Такая нормализация нужна потому, что матрицы $\hat{F}_{XL}$ с большой размерностью $N$ приводят к большим значениям в матрице Грама. Большие значения в свою очередь могут спровоцировать то, что первые сверточные слои (до применения pooling) будут иметь наибольший вклад в процесс градиентного спуска. Но стилевые признаки по большей части содержатся в глубоких слоях нейронной сети, поэтому эта нормализация важна.

In [ ]:
def gram_matrix(input):
    a, b, c, d = input.size()  # a=batch size(=1)
    # b=number of feature maps
    # (c,d)=dimensions of a f. map (N=c*d)

    features = input.view(a * b, c * d)  # resise F_XL into \hat F_XL

    G = torch.mm(features, features.t())  # compute the gram product

    # we 'normalize' the values of the gram matrix
    # by dividing by the number of element in each feature maps.
    return G.div(a * b * c * d)

Style loss, аналогично content loss, вычисляется через квадрат расстояния между $G_{XL}$ and $G_{SL}$.




In [ ]:
class StyleLoss(nn.Module):

    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()

    def forward(self, input):
        G = gram_matrix(input)
        self.loss = F.mse_loss(G, self.target)
        return input

Импортируем модель
-------------------

Будем использовать предобученную нейронную сеть. Для этой задачи возьмём VGG19.

VGG в имплементации pytorch имеет две части:
* ``Sequential`` modules: ``features`` (содержит свёртки и пулинги)
* ``Classifier`` (содержит полносвязные слои). 

Мы будем пользоваться модулем ``features``, так как нам нужны результаты применения свёрток для вычисления контентного и стилевого лоссов.

Так как мы используем предобученную нейронную сеть и дообучать мы её не планируем, то мы ставим сеть в режим применения: ``.eval()``.


In [ ]:
cnn = models.vgg19(pretrained=True).features.to(device).eval()

VGG сети обучены на нормализованных изображениях с параметрами mean=[0.485, 0.456, 0.406] и std=[0.229, 0.224, 0.225].
Поэтому перед подачей изображения в сеть его необходимо нормализовать таким же образом.



In [ ]:
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

# create a module to normalize input image so we can easily put it in a
# nn.Sequential
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        # .view the mean and std to make them [C x 1 x 1] so that they can
        # directly work with image Tensor of shape [B x C x H x W].
        # B is batch size. C is number of channels. H is height and W is width.
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std = torch.tensor(std).view(-1, 1, 1)

    def forward(self, img):
        # normalize img
        return (img - self.mean) / self.std

Теперь мы должны добавить к свёрточной части сети VGG19 стилевой и контентный лосс сразу после свёрточных слоёв.

Для этого необходимо создать новый модуль ``Sequential`` с добавленными в нужные места лоссами.



In [ ]:
# desired depth layers to compute style/content losses :
content_layers_default = ['conv_4']
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                               style_img, content_img,
                               content_layers=content_layers_default,
                               style_layers=style_layers_default):
    cnn = copy.deepcopy(cnn)

    # normalization module
    normalization = Normalization(normalization_mean, normalization_std).to(device)

    # just in order to have an iterable access to or list of content/syle
    # losses
    content_losses = []
    style_losses = []

    # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
    # to put in modules that are supposed to be activated sequentially
    model = nn.Sequential(normalization)

    i = 0  # increment every time we see a conv
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            # The in-place version doesn't play very nicely with the ContentLoss
            # and StyleLoss we insert below. So we replace with out-of-place
            # ones here.
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        if name in content_layers:
            # add content loss:
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            # add style loss:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)

    # now we trim off the layers after the last content and style losses
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break

    model = model[:(i + 1)]

    return model, style_losses, content_losses

Теперь выбираем input image. В качестве input image можно использовать, например, исходное (content) изображение или белый шум.



In [ ]:
input_img = content_img.clone()

# add the original input image to the figure:
plt.figure()
imshow(input_img, title='Input Image')

Градиентный спуск
----------------

Как предложил автор алгоритма, мы будем использовать алгоритм
L-BFGS градиентного спуска. 

В данной версии алгоритма мы обучаем не веса в сети, а исходное изображение, чтобы минимизировать content/style
losses. Для этого используем pytorch ``optim.LBFGS`` и передаем наше изображение (как тензор) в оптимизатор.




In [ ]:
def get_input_optimizer(input_img):
    # this line to show that input is a parameter that requires a gradient
    optimizer = optim.LBFGS([input_img.requires_grad_()])
    return optimizer

Наконец, создадим функцию, осуществляющую стилевой перенос.
* на каждой итерации она получает на вход изображение и вычисляет новые значения лоссов (style/content).
* также на каждом шаге с помощью ``backward`` методов динамически вычисляются градиенты функций потерь. 

Также необходимо привести значения тензоров к отрезку $[0;1]$ на каждой итерации.



In [ ]:
def run_style_transfer(cnn, normalization_mean, normalization_std,
                       content_img, style_img, input_img, num_steps=300,
                       style_weight=1000000, content_weight=1):
    """Run the style transfer."""
    print('Building the style transfer model..')
    model, style_losses, content_losses = get_style_model_and_losses(cnn,
        normalization_mean, normalization_std, style_img, content_img)
    optimizer = get_input_optimizer(input_img)

    print('Optimizing..')
    run = [0]
    while run[0] <= num_steps:

        def closure():
            # correct the values of updated input image
            input_img.data.clamp_(0, 1)

            optimizer.zero_grad()
            model(input_img)
            style_score = 0
            content_score = 0

            for sl in style_losses:
                style_score += sl.loss
            for cl in content_losses:
                content_score += cl.loss

            style_score *= style_weight
            content_score *= content_weight

            loss = style_score + content_score
            loss.backward()

            run[0] += 1
            if run[0] % 50 == 0:
                print("run {}:".format(run))
                print('Style Loss : {:4f} Content Loss: {:4f}'.format(
                    style_score.item(), content_score.item()))
                print()

            return style_score + content_score

        optimizer.step(closure)

    # a last correction...
    input_img.data.clamp_(0, 1)

    return input_img

Запускаем алгоритм!

In [ ]:
output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                            content_img, test_img, input_img, style_weight=500000)

plt.figure()
imshow(output, title='Output Image')

# sphinx_gallery_thumbnail_number = 4
plt.ioff()
plt.show()

#### Задание 1. 

Мы запустили алгоритм, используя в качестве input_image исходное изображение (content_image). Попробуйте в качестве input_image взять белый шум. Визуализируйте результат.

In [ ]:
#your code here (you can change an existing code)

#### Задание 2.
Загрузите другую стилевую картинку (например, "звездную ночь" Ван Гога или любую другую), не забудьте привести её к тем же размерам, что и картинка с содержанием.

Примените новый стиль к изображению. Нравится ли вам результат?

In [ ]:
#your code here (you can change an existing code)

#### Задание 3.
Поэкспериментируйте с коэффициентами при style_loss и content_loss (по дефолту используются значения style_weight=1000000, content_weight=1).

Выведите на экран итоговые изображения для различных по порядку style_weight (используйте значения весов в диапазоне от 0 до 1).

In [ ]:
#your code here

#### Задание 4.
Для вычисления итогового style_loss мы суммируем style_loss по первым пяти свёрточным слоям, то есть каждый слой (из первых пяти) имеет одинаковый вклад в итоговый style_loss. Но можно вычислять итоговый style_loss, суммируя style_loss'ы по слоям с весами. 

Создайте список весов style_weights и запишите туда пять весов, которые по вашему мнению дают наибольшие артефакты стиля. Измените вычисление style_loss в процедуре run_style_transfer, домножая style_loss с каждого слоя на соответствующий вес. 

Проведите эксперименты для:
* вашего массива style_weights
* style_weights = [1, 0, 0, 0, 0]
* style_weights = [0, 0, 0, 0, 1]

Какие выводы можно сделать из этих экспериментов?


In [ ]:
#your code here

#### Задание 5.
Описанный в ноутбуке алгоритм работает достаточно медленно, особенно на изображениях большого размера. Как можно модифицировать алгоритм, чтобы снизить число обучаемых параметров?

In [ ]:
#your answer here